# Generative example

This example uses the LangChain framework with Ollama to generate synthetic clinical notes from synthetic structured data.

Running this notebook requires an additional install of [Ollama](https://ollama.ai/) and the particular model used is `llama2:latest` (also named `llama2:7b-chat`) from the [Ollama model library](https://ollama.ai/library/llama2/tags).

Open a terminal and run `ollama pull llama2` to download the model.

## Import libraries

In [1]:
import json

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

## Load input in batch

In [2]:
input_path = "../../data/synthea_dataset.json"

with open(input_path) as file:
    data = json.load(file)

batch = []

for i in range(5):
    batch.append({"data": json.dumps(data[i])})

## Load LLM

The model used to run inference can be easily swapped out using LangChain and Ollama!

Simply open a terminal and run `ollama pull <model_name:tag>` to retrieve any model from the Ollama model library and pass `<model_name:tag>` as the new argument when instantiating the LLM.

For example, if you wanted to use Mistral instead of Llama2, you would need to run `ollama pull mistral` in a terminal and set `model="mistral"` below.

In [3]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Ollama(model="llama2", callback_manager=callback_manager)

## Construct prompt

In [4]:
template = """[INST]
<<SYS>>
You are a medical student answering an exam question about writing clinical notes for patients.
<</SYS>>

Keep in mind that your answer will be asssessed based on incorporating all the provided information and the quality of prose.

1. Use prose to write an example clinical note for this patient's doctor.
2. Use less than three sentences.
3. Do not provide a diagnosis or recommendations.
4. Use the following information:

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm

## Run inference

In [5]:
outputs = chain.batch(batch)

Here is an example clinical note for Mr. Brandon McLaughlin:

"Mr. McLaughlin presents with acute bronchitis, diagnosed based on symptoms of cough and fever. His medical history includes asthma and allergies. He is a single male, 58 years old, and his address is 649 Schaden Estate Suite 18, Southampton, SO15 9UN. His NHS number is 568 968 0803. The care plan includes respiratory therapy to help manage symptoms."Clinical Note:
Mr. Esteban Altenwerth presented to the clinic on 10/29/2005 with symptoms of acute bacterial sinusitis. He is a married male, aged 20, with a history of chronic sinusitis. His NHS number is 201 826 4805 and his address is 1019 Hane Brook, Basingstoke, RG21 8DW. He reports a sudden onset of fever, headache, and nasal congestion. Provider: Dr. Mercy Kuhlman at Old Basing Health Centre.Mr. Ervin O'Connell, an 82-year-old male patient, presented to Ringwood Medical Centre on February 27, 1996, with a chief complaint of cough and chest congestion for the past week. Hi